In [1]:
import pandas as pd
pd.options.display.max_columns = 50
import sqlite3
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from data_utils import (
    clean_transcripts,
    flesch,
    fog,
    smog,
    terms,
    uniqueterms, 
    ttr, 
    mtld, 
    hdd
)

con = sqlite3.connect("../data/cnn.db")

In [2]:
df_cnn = (pd.read_sql_query("SELECT * from cnn_stories", con)
          .rename(columns={'publish_date': 'date'})
          .query('date==date')
          .assign(text=lambda df: df.text.str.strip())
          .query('text!=""')
          .assign(
              text=lambda df: df.text.apply(clean_transcripts),
              # Compute readability
              flesch_reading_ease=lambda df: df.text.apply(flesch),
              fog=lambda df: df.text.apply(fog), 
              smog=lambda df: df.text.apply(smog),  
              terms=lambda df: df.text.apply(terms),
              # Compute lexical richness
              uniqueterms=lambda df: df.text.apply(uniqueterms), 
              ttr=lambda df: df.text.apply(ttr),
              mtld=lambda df: df.text.apply(mtld),
              hdd=lambda df: df.text.apply(hdd),
         )
         )
        

df_cnn.head(3)

,source,url,date,title,authors,text,flesch_reading_ease,fog,smog,terms,uniqueterms,ttr,mtld,hdd
0,CNN,https://www.cnn.com/interactive/2022/07/politi...,None,Power in chaos: Steve Bannon is disrupting dem...,[],"as a single mom with a full time job, wendy ah...",47.287335,13.401459,16.973005,7640,2133,0.294085,123.178881,0.875235
1,CNN,https://www.cnn.com/2022/07/18/politics/trumpi...,2022-07-18 00:00:00,The truth about 2020 may not stop a Trump resu...,"[""Analysis Stephen Collinson""]",the ex president's dereliction of duty that fa...,38.450361,17.429995,19.933966,1636,639,0.389634,113.497553,0.850355
2,CNN,https://www.cnn.com/2022/07/18/politics/januar...,2022-07-18 00:00:00,The January 6 committee is returning to primet...,"[""Analysis Paul Leblanc""]",a version of this story appears in cnn's what ...,55.688457,NaN,14.118598,233,151,0.637131,121.786725,0.853515


In [3]:
(df_cnn
 .drop('text', axis=1)
 .to_csv('../data/cnn_2_textstat.csv.gz', compression="gzip")
)